In [1]:
param_search_query = 'Turbocharger'
param_len = 12

list_url = ['https://rotomaster.com/search.php?section=product&_bc_fsnf=1&Part+Type=Turbocharger&limit=30&sort=alphaasc&search_query=' + param_search_query + '&page=' + str(_+1) for _ in range(param_len)]

print('总数量：' + str(len(list_url)))
print()

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =

                html = etree.HTML(str(soup))

                list_url = html.xpath('//h4[@class="card-title"]/a/@href')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Menu': url,
                                        'Url': list_url})
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Menu': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./menu_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：12

https://rotomaster.com/search.php?section=product&_bc_fsnf=1&Part+Type=Turbocharger&limit=30&sort=alphaasc&search_query=Turbocharger&page=3  <->  [ok] - 剩余数量：0 - 当前时间：18:55:35
https://rotomaster.com/search.php?section=product&_bc_fsnf=1&Part+Type=Turbocharger&limit=30&sort=alphaasc&search_query=Turbocharger&page=10  <->  [ok] - 剩余数量：0 - 当前时间：18:55:35
https://rotomaster.com/search.php?section=product&_bc_fsnf=1&Part+Type=Turbocharger&limit=30&sort=alphaasc&search_query=Turbocharger&page=2  <->  [ok] - 剩余数量：0 - 当前时间：18:55:35
https://rotomaster.com/search.php?section=product&_bc_fsnf=1&Part+Type=Turbocharger&limit=30&sort=alphaasc&search_query=Turbocharger&page=5  <->  [ok] - 剩余数量：0 - 当前时间：18:55:35
https://rotomaster.com/search.php?section=product&_bc_fsnf=1&Part+Type=Turbocharger&limit=30&sort=alphaasc&search_query=Turbocharger&page=6  <->  [ok] - 剩余数量：0 - 当前时间：18:55:35
https://rotomaster.com/search.php?section=product&_bc_fsnf=1&Part+Type=Turbocharger&limit=30&sort=alphaasc&sear